In [569]:
import warnings
warnings.filterwarnings("ignore")
import glob
import pandas as pd
import numpy as np
from random import sample,seed,shuffle
np.random.seed(42)
random_state = np.random.RandomState(42)
seed(a=42)

files = glob.glob("/Users/javie/OneDrive/Documentos/CIMAT/Sem 3/Preprocesamiento/Tarea 2/csv/*.csv")
Data_frames=[]
for file in files:    
    Data_frames.append(pd.read_csv(file))
Dfs5  = list( Data_frames[i] for i in [3,7,11])
Dfs10 = list( Data_frames[i] for i in [0,4,8])
Dfs15 = list( Data_frames[i] for i in [1,5,9])
Dfs20 = list( Data_frames[i] for i in [2,6,10])


In [638]:
from numpy import zeros, sum as sumnp
from scipy.stats import mode
from sklearn.base import BaseEstimator
from sklearn.utils import check_X_y, check_array
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestCentroid
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from imblearn.pipeline  import Pipeline
from sklearn.preprocessing import StandardScaler
import scipy.stats as ss

class NoiseHandling(BaseEstimator):
    def __init__(self, method = 'ENN', n_neighbors = 1, n_splits = 5, 
                 filter_type = 'majority', random_state = None):
        self.method = method
        self.n_neighbors = n_neighbors
        self.n_splits = n_splits
        self.filter_type = filter_type
        self.random_state = random_state
        
    def __filtering_rule(self, y_pred, y):
        if self.filter_type == 'majority':
            y_pred_mode = mode(y_pred, axis = 1)[0].ravel()
            return y_pred_mode == y
        elif self.filter_type == 'consensus':
            y_pred_bool = y_pred == y[:, None]
            return sumnp(y_pred_bool, axis = 1) > 0
        else:
            raise ValueError('Undefined rule')

    #Iterative edited-K Nearest Centroid Neighborhood
    def __iekncn_fit(self, X, y):
        f = True
        y=pd.DataFrame(y)
        y_aux=y

        while f == True:
            clf = NearestCentroid()
            clf.fit(X, y_aux.values.ravel())
            labels = clf.predict(X)
            y_pred =labels.ravel()

            if (labels==y_aux.values.ravel()).sum()!=len(y_aux):
                X=X[labels==y_aux.values.ravel()]
                y_aux=y_aux[labels==y_aux.values.ravel()]
            else:
                f=False
        indexes = np.arange(len(y))
        j = 0
        Filter=[]

        for i in range(len(indexes)):
            if indexes[i]==y.index[j]:
                Filter.append(True)
                j=j+1
            else:
                Filter.append(False)
        self.filter_ = Filter
        return self
            
            
    #Complementary Neural Network
    def __cmtnn_fit(self,X,y):  
        
        y_comp=y
        shuffle(y_comp)
        mt = MLPClassifier(random_state=1, max_iter=500).fit(X,y)
        mc = MLPClassifier(random_state=1, max_iter=500).fit(X,y_comp)
        y_t = mt.predict(X)
        y_c = mc.predict(X)        
        self.filter_ = (y!=y_t)&(y!=y_c)
        
        return self

    #Decremental Reduction Optimization Procedure
    def __drop_fit(self, X, y):
        S = X
        nn_search = NearestNeighbors(n_neighbors = 5 + 1)
        nn_search.fit(S)
        neigh_ind = nn_search.kneighbors(S, return_distance = False)
        Filter=[]

        for i in range(np.shape(neigh_ind)[0]):
            #associates.append(X[neigh_ind[i,1:]])
            neigh = KNeighborsClassifier(n_neighbors=5+1)
            neigh.fit(X, y)
            ni =  (neigh.predict(X[neigh_ind[i,:]])==y[neigh_ind[i,:]]).sum()
            ne =  (neigh.predict(X[neigh_ind[i,1:]])==y[neigh_ind[i,1:]]).sum()
            Filter.append(ni<=ne)
        self.filter_ = list(map(operator.not_, Filter))
        return self
    
    # Cross Validates Commites Filter 
    def __cvcf_fit(self,X,y):
        skf = StratifiedKFold(n_splits = self.n_splits, shuffle = True,
                              random_state = self.random_state)
        predictions = zeros((X.shape[0], 3))
        for train_idx, test_idx in skf.split(X, y):
            X_train, y_train = X[train_idx], y[train_idx]
            X_test = X[test_idx]
            
            model = DecisionTreeClassifier(random_state=42)
            model.fit(X_train, y_train)
            predictions = model.predict(X)
            
        self.filter_ = predictions == y
        return self   
    
    #Edited Nearest Neighbor
    def __enn_fit(self, X, y):
        nn_search = NearestNeighbors(n_neighbors = self.n_neighbors + 1)
        nn_search.fit(X)
        neigh_ind = nn_search.kneighbors(X, return_distance = False)[:, 1:]
        labels = y[neigh_ind]
        y_pred = mode(labels, axis = 1)[0].ravel()
        self.filter_ = y == y_pred
        return self
    
    #Ensemble Filter
    def __ef_fit(self, X, y):
        skf = StratifiedKFold(n_splits = self.n_splits, shuffle = True,
                              random_state = self.random_state)
        predictions = zeros((X.shape[0], 3))
        for train_idx, test_idx in skf.split(X, y):
            X_train, y_train = X[train_idx], y[train_idx]
            X_test = X[test_idx]
            
            learning_algorithms = [DecisionTreeClassifier(), \
                                  KNeighborsClassifier(), \
                                  LogisticRegression()]
            for index, model in enumerate(learning_algorithms):
                model.fit(X_train, y_train)
                predictions[test_idx, index] = model.predict(X_test)
                
        self.filter_ = self.__filtering_rule(predictions, y)
        return self 
    
    def __filter_resample(self, X, y):
        return X[self.filter_], y[self.filter_]
            
    def __resample(self, X, y):
        X = check_array(X)
        
        if (self.method == 'ENN') or (self.method == 'EF') or (self.method == 'IEKNCN') or (self.method == 'DROP') or (self.method == 'CMTNN') or (self.method == 'CVCF'):
            return self.__filter_resample(X, y)
        else:
            raise ValueError('Undefined method')
    
    def fit(self, X, y):
        X, y = check_X_y(X, y)
        
        if self.method == 'ENN':
            return self.__enn_fit(X, y)
        elif self.method == 'EF':
            return self.__ef_fit(X, y)
        elif self.method == 'IEKNCN':
            return self.__iekncn_fit(X, y)
        elif self.method == 'DROP':
            return self.__drop_fit(X, y)        
        elif self.method == 'CMTNN':
            return self.__cmtnn_fit(X, y)
        elif self.method == 'CVCF':
            return self.__cvcf_fit(X, y)
        else:
            raise ValueError('Undefined method')
            
    def fit_resample(self, X, y):
        return self.fit(X, y).__resample(X, y)

def classification(X,y,X_train,y_train,X_test,y_test,clf,method):
    classifier = clf
    y_score = classifier.fit(X_train, y_train).predict(X_test)
    pipe = Pipeline([('standardize', StandardScaler()),
                  ('noise_handler', NoiseHandling(method = method, filter_type = 'consensus',n_neighbors = 5)),
                  ('classifier', classifier)])
    pipe.fit(X_train, y_train)
    score=pipe.score(X_test, y_test)
    return score
        
def scores_rsfk(X,y,clf):
    rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5)
    scores1,scores2,scores3,scores4,scores5=[],[],[],[],[]
    for train_index, test_index in rskf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #Classifiers "ENN","EF","IEKNCN","DROP","CVCF"
        scores1.append(classification(X,y,X_train,y_train,X_test,y_test,clf,"ENN"))
        scores2.append(classification(X,y,X_train,y_train,X_test,y_test,clf,"EF"))
        scores3.append(classification(X,y,X_train,y_train,X_test,y_test,clf,"IEKNCN"))
        scores4.append(classification(X,y,X_train,y_train,X_test,y_test,clf,"DROP"))
        scores5.append(classification(X,y,X_train,y_train,X_test,y_test,clf,"CVCF"))
        
    scores = np.array([scores1,scores2,scores3,scores4,scores5]).T
    return scores

def statistics(df,clf):
    X = df.drop('Class', 1).to_numpy()
    y = df['Class']
    scores=scores_rsfk(X,y,clf)
    return sp.posthoc_nemenyi_friedman(scores).mean()


In [654]:
df=Dfs5
clf1 = DecisionTreeClassifier(random_state=42)
clf2 = SVC(kernel='linear', probability=True,max_iter=1000)
clf3 = KNeighborsClassifier(4)
classifiers=[clf1,clf2,clf3]
#table = pd.DataFrame([], columns=["ENN", "EF","IEKNCN","DROP","CVCF"])
p_values=[]
for i in range(3):    
    for clf in classifiers:
        p_values.append(statistics(df[i],clf))


IndexError: list index out of range

In [724]:
bckp=p_values
np.shape(bckp)
np.reshape(bckp,(9,5))

array([[0.86054911, 0.64926359, 0.7247668 , 0.86530605, 0.8653071 ],
       [0.92      , 0.92      , 0.92      , 0.92      , 0.92      ],
       [0.83621458, 0.88462739, 0.91673139, 0.88462739, 0.76220075],
       [0.56313852, 0.49069361, 0.36490698, 0.56313852, 0.43491354],
       [0.71867905, 0.36056213, 0.7196697 , 0.52200686, 0.7227285 ],
       [0.76476467, 0.32344034, 0.68374919, 0.75023887, 0.75924862],
       [0.29108424, 0.47088424, 0.53178246, 0.41877769, 0.50829345],
       [0.75177364, 0.81618942, 0.8653065 , 0.87660124, 0.6592578 ],
       [0.32849345, 0.4173753 , 0.40735192, 0.44301685, 0.44424855]])

In [728]:
aux5=[[0.86054911, 0.64926359, 0.7247668 , 0.86530605, 0.8653071 ],
       [0.92      , 0.92      , 0.92      , 0.92      , 0.92      ],
       [0.83621458, 0.88462739, 0.91673139, 0.88462739, 0.76220075],
       [0.56313852, 0.49069361, 0.36490698, 0.56313852, 0.43491354],
       [0.71867905, 0.36056213, 0.7196697 , 0.52200686, 0.7227285 ],
       [0.76476467, 0.32344034, 0.68374919, 0.75023887, 0.75924862],
       [0.29108424, 0.47088424, 0.53178246, 0.41877769, 0.50829345],
       [0.75177364, 0.81618942, 0.8653065 , 0.87660124, 0.6592578 ],
       [0.32849345, 0.4173753 , 0.40735192, 0.44301685, 0.44424855]]
aux10=[[0.46522957, 0.58512103, 0.44482127, 0.57712187, 0.45422009],
       [0.88462739, 0.92      , 0.84598617, 0.88462739, 0.91673139],
       [0.78509994, 0.6082013 , 0.55683022, 0.81388171, 0.81715014],
       [0.41183406, 0.48235078, 0.42295374, 0.49629718, 0.47961465],
       [0.44161853, 0.82748416, 0.7764975 , 0.7951724 , 0.80246447],
       [0.74081904, 0.62425312, 0.80585562, 0.52384395, 0.79782985],
       [0.32218795, 0.42345201, 0.50675668, 0.38715632, 0.54403428],
       [0.63802546, 0.61395303, 0.7840854 , 0.61838312, 0.50654095],
       [0.32849345, 0.41469992, 0.41207372, 0.44277672, 0.43661123]]
aux15 =[[0.27099899, 0.58305305, 0.58305305, 0.37122395, 0.61542595],
       [0.5174993 , 0.7951724 , 0.82748416, 0.84449729, 0.81034544],
       [0.32849345, 0.42129195, 0.4033066 , 0.44336541, 0.45255908],
       [0.55576839, 0.69823867, 0.48557244, 0.59849724, 0.48557244],
       [0.8653071 , 0.64926359, 0.86054911, 0.7247668 , 0.86530605],
       [0.47268921, 0.49954643, 0.59905679, 0.55602896, 0.69850932],
       [0.6091385 , 0.59301814, 0.46742738, 0.53782118, 0.36860383],
       [0.76000691, 0.68424785, 0.75769925, 0.77676369, 0.8251765 ],
       [0.53962729, 0.62908042, 0.63518592, 0.73484274, 0.71867905]]
aux20 =[[0.22177857, 0.47614136, 0.67802682, 0.58674571, 0.67410683],
       [0.40003733, 0.81715032, 0.78217939, 0.78217939, 0.72383428],
       [0.32849345, 0.43781551, 0.38261274, 0.44529728, 0.46911889],
       [0.56226718, 0.56288007, 0.38952732, 0.56571403, 0.38133396],
       [0.83320214, 0.75130223, 0.73484274, 0.64967404, 0.74967306],
       [0.75744321, 0.56562242, 0.69271208, 0.78340687, 0.76694668],
       [0.66163253, 0.62867988, 0.6666049 , 0.7914325 , 0.49514001],
       [0.92      , 0.92      , 0.92      , 0.92      , 0.92      ],
       [0.41794778, 0.67891839, 0.64632382, 0.7628591 , 0.64632382]]

In [730]:
aux=aux15
aux.append(np.reshape(aux,(9,5)).mean(axis=0))
aux=np.around(aux,3)
table = pd.DataFrame(aux, columns=["ENN", "EF","IEKNCN","DROP","CVCF"])
table.index=["Decision Tree","Decision Tree","Decision Tree","SVC","SVC","SVC", "KNC","KNC","KNC","Promedio"]
table["Database"]=["Heart","Heart","Heart","Pima","Pima","Pima","Ring","Ring","Ring",""]
table
print(table.to_latex())
ranks=[]
for i in range(9):    
    ranks.append(list(ss.rankdata(aux[i])))
print(np.around(np.array(ranks).mean(axis=0),2))

\begin{tabular}{lrrrrrl}
\toprule
{} &    ENN &     EF &  IEKNCN &   DROP &   CVCF & Database \\
\midrule
Decision Tree &  0.271 &  0.583 &   0.583 &  0.371 &  0.615 &    Heart \\
Decision Tree &  0.517 &  0.795 &   0.827 &  0.844 &  0.810 &    Heart \\
Decision Tree &  0.328 &  0.421 &   0.403 &  0.443 &  0.453 &    Heart \\
SVC           &  0.556 &  0.698 &   0.486 &  0.598 &  0.486 &     Pima \\
SVC           &  0.865 &  0.649 &   0.861 &  0.725 &  0.865 &     Pima \\
SVC           &  0.473 &  0.500 &   0.599 &  0.556 &  0.699 &     Pima \\
KNC           &  0.609 &  0.593 &   0.467 &  0.538 &  0.369 &     Ring \\
KNC           &  0.760 &  0.684 &   0.758 &  0.777 &  0.825 &     Ring \\
KNC           &  0.540 &  0.629 &   0.635 &  0.735 &  0.719 &     Ring \\
Promedio      &  0.547 &  0.617 &   0.624 &  0.621 &  0.649 &          \\
\bottomrule
\end{tabular}

[2.28 2.61 2.78 3.56 3.78]
